# Домашнее задание 3
**Продвинутое машинное обучение**

In [294]:
import nltk
import numpy as np
import random

from collections import Counter
from nltk.util import ngrams
from random import shuffle

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Loading and preparing data

In [295]:
WAR_AND_PEACE_PATH = '/content/drive/MyDrive/adv-ml/WarAndPeace.txt'
WAR_AND_PEACE_ENG_PATH = '/content/drive/MyDrive/adv-ml/WarAndPeaceEng.txt'
ANNA_KARENINA_PATH = '/content/drive/MyDrive/adv-ml/AnnaKarenina.txt'

In [296]:
with open(WAR_AND_PEACE_PATH, 'r') as file:
    war_and_peace = file.read().replace('\n', ' ')

with open(WAR_AND_PEACE_ENG_PATH, 'r') as file:
    war_and_peace_eng = file.read().replace('\n', ' ')

with open(ANNA_KARENINA_PATH, 'r') as file:
    anna_karenina = file.read().replace('\n', ' ')

In [297]:
text_ru = ' '.join([war_and_peace, anna_karenina])

In [298]:
example_texts = ['В то время как у Ростовых танцевали в зале шестой англез под звуки от усталости фальшививших музыкантов и усталые официанты и повара готовили ужин, с графом Безуховым сделался шестой уже удар. Доктора объявили, что надежды к выздоровлению нет; больному дана была глухая исповедь и причастие; делали приготовления для соборования, и в доме была суетня и тревога ожидания, обыкновенные в такие минуты. Вне дома, за воротами, толпились, скрываясь от подъезжающих экипажей, гробовщики, ожидая богатого заказа на похороны графа.',
                 'Пьер, решившись во всем повиноваться своей руководительнице, направился к диванчику, который она ему указала. Как только Анна Михайловна скрылась, он заметил, что взгляды всех бывших в комнате больше чем с любопытством и с участием устремились на него. Он заметил, что все перешептывались, указывая на него глазами, как будто со страхом и даже с подобострастием.',
                 'Княжна Марья возвратилась в свою комнату с грустным, испуганным выражением, которое редко покидало ее и делало ее некрасивое, болезненное лицо еще более некрасивым, села за свой письменный стол, уставленный миниатюрными портретами и заваленный тетрадями и книгами. Княжна была столь же беспорядочна, как отец ее порядочен. Она положила тетрадь геометрии и нетерпеливо распечатала письмо.']

In [299]:
RU_ABC = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '

## 1. Базовый частотный метод по Шерлоку Холмсу
Задачи:
* подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
* возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
* расшифруйте их таким частотным методом.

In [300]:
def process_text(text: str) -> str:
  text = text.lower()
  text = ''.join([char for char in text if char in RU_ABC])
  words = nltk.word_tokenize(text)
  return ' '.join(words)

def encrypt_text(text: str, key: list, alpha_key: list) -> str:
  return ''.join([key[alpha_key.index(char)] for char in text])

def descending_key(key_freq: Counter) -> list:
  return [v[0] for v in key_freq.most_common()]

def decrypt_text(text: str, corpus_freq: Counter) -> str:
  text_freq = Counter(text)
  text_freq = descending_key(text_freq)
  corpus_freq = descending_key(corpus_freq)
  text_freq.extend(list(set(corpus_freq) - set(text_freq)))

  return ''.join([corpus_freq[text_freq.index(char)] for char in text])

def get_accuracy(original_text: str, decoded_text: str) -> float:
  text_len = len(original_text)
  assert text_len == len(decoded_text), 'length of texts must be the same'
  overlaps = [1 for i in range(text_len) if original_text[i] == decoded_text[i]]
  return sum(overlaps) / text_len

In [301]:
text_ru_cleaned = process_text(text_ru)
print(text_ru_cleaned[:200])

война и мир самый известный роман льва николаевича толстого как никакое другое произведение писателя отражает глубину его мироощущения и философииэта книга из разряда вечных потому что она обо всем о 


In [302]:
# Частотность букв по корпусам
corpus_freq = Counter(text_ru_cleaned)
print(corpus_freq)

Counter({' ': 385114, 'о': 223691, 'е': 166169, 'а': 162313, 'н': 133258, 'и': 129712, 'т': 115258, 'с': 103252, 'л': 98191, 'в': 91386, 'р': 80859, 'к': 67788, 'д': 58019, 'м': 56467, 'у': 53583, 'п': 47940, 'я': 42921, 'ь': 38351, 'г': 36870, 'ы': 36447, 'б': 34028, 'з': 32723, 'ч': 31207, 'ж': 21480, 'й': 21072, 'ш': 17158, 'х': 15586, 'ю': 12307, 'э': 6648, 'ц': 6172, 'щ': 5568, 'ф': 2990, 'ъ': 695, 'ё': 462})


In [303]:
example_text_1 = process_text(example_texts[0])
example_text_2 = process_text(example_texts[1])
example_text_3 = process_text(example_texts[2])

In [304]:
# Делаем ключ для шифрования текста
random.seed(42)
key = list(RU_ABC)
shuffle(key)
alpha_key = descending_key(corpus_freq)

In [305]:
print(key)
print(alpha_key)

['е', 'т', 'у', 'л', 'ч', 'ы', 'о', 'й', 'п', 'к', 'щ', 'н', 'ш', 'и', 'з', 'ь', 'ё', 'х', 'а', 'я', 'м', 'с', 'в', 'ю', 'ъ', 'ф', 'г', 'ц', 'д', 'э', ' ', 'р', 'б', 'ж']
[' ', 'о', 'е', 'а', 'н', 'и', 'т', 'с', 'л', 'в', 'р', 'к', 'д', 'м', 'у', 'п', 'я', 'ь', 'г', 'ы', 'б', 'з', 'ч', 'ж', 'й', 'ш', 'х', 'ю', 'э', 'ц', 'щ', 'ф', 'ъ', 'ё']


In [306]:
# Зашифрованные примеры текстов
ciphertext_1 = encrypt_text(example_text_1, key, alpha_key)
ciphertext_2 = encrypt_text(example_text_2, key, alpha_key)
ciphertext_3 = encrypt_text(example_text_3, key, alpha_key)

In [327]:
print(f'Зашифрованный текст №1: \n{ciphertext_1}')
print(f'Зашифрованный текст №2: \n{ciphertext_2}')
print(f'Зашифрованный текст №3: \n{ciphertext_3}')

Зашифрованный текст №1: 
кеотекщуиёенлнезещтйоткягеолчэуклпыекеслпуефуйотъелчапусеьтшескзныетоезйолптйоыерлпхфыкыкфыгеизсянлчоткеыезйолпяуетрыэылчояеыеьтклщлеатоткыпыезюычейеащлртиемусзгткяиейшуплпйёефуйотъезюуезшлщештнотщлетмбёкыпыевотечлшуюшяенекясштщткпучыцечуоемтпхчтизешлчлемяплеапзглёеыйьткушхеыеьщывлйоыуешуплпыеьщыатоткпучыёешпёейтмтщтклчыёеыекештиуемяплейзуочёеыеощукталетюышлчыёетмянчткуччяуекеолныуеиычзояекчуештилеслектщтолиыеотпьыпыйхейнщяклёйхетоеьтшбусюлц ыгедныьлюуъеащтмтк ыныетюышлёемталотатеслнлслечлеьтгтщтчяеащлрл
Зашифрованный текст №2: 
ьхущещуфыкфыйхектекйуиеьткычтклохйёейктуъещзнтктшыоупхчыэуечльщлкыпйёенешыклчвынзентотщяъетчлеуизезнлслпленлнеотпхнтелччлеиыглъпткчлейнщяплйхетчеслиуоыпевотексапёшяекйугемякфыгекентичлоуемтпхфуевуиейепцмтьяойоктиеыейезвлйоыуиезйощуиыпыйхечлечуатетчеслиуоыпевотекйуеьущуфуьояклпыйхезнлсяклёечлечуатеаплслиыенлнемзшотейтейощлгтиеыешлюуейеьтштмтйощлйоыуи
Зашифрованный текст №3: 
нчёючлеилщхёектскщлоыплйхекейктцентичлозейеащзйочяиеыйьзалччяие

In [308]:
# Расшифрованные тексты
decrypt_text_1 = decrypt_text(ciphertext_1, corpus_freq)
decrypt_text_2 = decrypt_text(ciphertext_2, corpus_freq)
decrypt_text_3 = decrypt_text(ciphertext_3, corpus_freq)

In [309]:
print(f'Расшифрованный текст №1: \n{decrypt_text_1} \nсовпадение символов: {round(get_accuracy(example_text_1, decrypt_text_1), 2)}')
print(f'Расшифрованный текст №2: \n{decrypt_text_2} \nсовпадение символов: {round(get_accuracy(example_text_2, decrypt_text_2), 2)}')
print(f'Расшифрованный текст №3: \n{decrypt_text_3} \nсовпадение символов: {round(get_accuracy(example_text_3, decrypt_text_3), 2)}')

Расшифрованный текст №1: 
н то нвигп уеу м вортондч тесюинела н яели йиртох есьлия ыок янмуа от мртелорта шелжйананйач гмядуестон а мртелди ошаюаестд а ыоневе ьотонала мзас р ьвешог биямчондг ркилелрп йиртох мзи мкев коутове обэпнала цто секизкд у ндяковонлисащ сит болжсогм кесе бдле ьлмчеп арыоникж а ывацертаи килела ываьотонлисап клп робовонесап а н коги бдле рмитсп а твиноье озакесап обдусониссди н теуаи гасмтд нси коге яе новотега толыаларж рувднепрж от ыокэиязещфач ъуаыезих ьвобонфауа озакеп боьетоьо яеуеяе се ыочовосд ьвеше 
совпадение символов: 0.35
Расшифрованный текст №2: 
пмад даыитыинм те тнар петилетосмнч нтеаш дукетеьисавмлиха лопдотивнч к ьитолбику кеседяш ело ару укогово кок севмке олло рижошветло нкдявонм ел горасив бсе тгйвчья тнаж зятыиж т керлоса зевмыа бар н вюзепяснстер и н убонсиар унсдаривинм ло лайе ел горасив бсе тна падаыапсятовинм укогяточ ло лайе йвогори кок зуьсе не нсдожер и ьоэа н пеьезенсдонсиар 
совпадение символов: 0.29
Расшифрованный текст №3: 
княгне

**Вывод:** Как видно по качеству текста и по полученному скору - данный метод не эффективный.

## 2. Пункт 1 с биграмамми
Задачи:
* подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
* проведите тестирование аналогично п.1, но при помощи биграмм

In [310]:
def decrypt_text_with_ngrams(text: str, n_gram: int, origin_ngram_freq: Counter) -> str:
  text_n_gram_freq = Counter(ngrams(list(text), n_gram))
  text_n_gram_freq = descending_key(text_n_gram_freq)
  origin_ngram_freq = descending_key(origin_ngram_freq)
  text_n_gram_freq.extend(list(set(origin_ngram_freq) - set(text_n_gram_freq)))
  end = ''.join(origin_ngram_freq[text_n_gram_freq.index(tuple(text[-2:]))])[1]
  decrypted_text = ''.join([origin_ngram_freq[text_n_gram_freq.index(tuple(text[i:i+2]))][0] 
                  for i in range(len(text)) if i < len(text) - 1]) + end
  return decrypted_text

In [311]:
# Частотность биграм по корпусам и примера текста с лучшим скором из пункта 1
n_gram = 2
text_ngram_freq = Counter(ngrams(list(text_ru_cleaned), n_gram))
example_text_ngram_freq = Counter(ngrams(list(example_text_3), n_gram))

In [312]:
print(f'Количество биграм в оригинальных корпусах: {len(text_ngram_freq)}')
print(f'Количество биграм в тексте: {len(example_text_ngram_freq)}')

Количество биграм в оригинальных корпусах: 854
Количество биграм в тексте: 170


In [313]:
decrypted_text = decrypt_text_with_ngrams(ciphertext_3, n_gram, text_ngram_freq)

In [314]:
# Полученный результат
decrypted_text

'влвр озсиети ядаееглн пру ю тя о осв гчм ота  кс окнпонсосс ок лнема  хмк окл  тароикоьт эиммооьнланитте ооьнлаейитепдояталанранй сатаресвьныеаланрлаейитепдолок ае ооыо тяе твправ с в  клнси к дор с в заи  гдшх оаит  чеон  аиниоыдпор с в амнеечеааини в но аи влвр ол б о клнбуеаалзоо    бжс о идлекмиинлат    бжс сед от ношун оамнеечиутиусвнеодиниейнмег ееояьрепо отг о отвпракр'

In [315]:
print(f'Совпадение букв при использовании биграмм: {round(get_accuracy(example_text_3, decrypted_text), 2)}')

Совпадение букв при использовании биграмм: 0.07


**Вывод:** данный метод показывает результат хуже чем частотный метод по символам, что не удивительно — большая разница между количеством биграмм полученных из корпусов и примера текста, из за чего:
1. Проблематично сопоставить частотное распределение биграмм, так как в коротком тексте их слишком мало.
2. Появляется больше ошибок из за несоответсвия частотного распредления биграмм.

## 3. Метод обучения на основе MCMC-сэмплировании
Задачи:
* предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
* реализуйте и протестируйте его, убедитесь, что результаты улучшились.

**Комментарий:**
Можем рассмотреть текст поделенный на биграммы как Марковскую цепь, где вероятностями перехода по цепи и будут частоты биграмм.

Для обучения перестановки ключа шифрования будем считать вероятность пораждения заданного текста как произведение вероятностей биграмм заданного текста.

Так как количество возможных перестановок порядка 10 в 34 степени, то для оптимизации воспользуемся MCMC-сэмплированием.

Алгоритм действий:
1. Выбираем рандомный ключ шифрования.
2. Расшифровываем текст с текущим ключом и новым ключем с одной перестановкой.
3. Используя скоринговую функцию высчитываем скоры для результатов с использованием текущего и нового ключа.
4. Если скор с новым ключом лучше, то делаем его текущим.
5. В противном случае "подбрасываем монетку" (число от 0 до 1) и если оно меньше отношения скора с новым ключом к скору с текущим ключом, то делаем новый ключ текущим.
6. Повторяем с шага 2.

In [367]:
def decrypt_text_with_key(text: str, key: list,  alpha_key: list) -> str:
  key.extend(list(set(alpha_key) - set(key)))
  return ''.join([alpha_key[key.index(char)] for char in text])

def get_score(text: str,
               key: list, 
               alpha_key: list, 
               n_gram: int, 
               corpus_ngram_freq: Counter) -> float:
  score = 0
  decrypted_text = decrypt_text_with_key(text, key, alpha_key)
  decrypted_text_ngram_freq = Counter(ngrams(list(decrypted_text), n_gram))
  for ngram in decrypted_text_ngram_freq:
    if ngram not in corpus_ngram_freq:
      score += decrypted_text_ngram_freq[ngram] * np.log(0.75)
      continue
    score += decrypted_text_ngram_freq[ngram] * np.log(corpus_ngram_freq[ngram])
  return score

def update_key(key: list) -> list:
  new_key = key.copy()
  up_val = len(key)
  a_idx = np.random.randint(-1, up_val)
  b_idx = np.random.randint(-1, up_val)
  if a_idx == b_idx:
    return update_key(new_key)
  new_key[a_idx], new_key[b_idx] = new_key[b_idx], new_key[a_idx]
  return new_key

def mcmc_decrypting(ciphertext: str,
                    alpha_key: list, 
                    n_gram: int, 
                    text_ngram_freq: Counter,
                    tries: int,
                    iters: int) -> float:
  best_key = None
  best_score = 0

  for _ in range(tries):

    cur_key = descending_key(Counter(ciphertext))

    for _ in range(iters):
      new_key = update_key(cur_key)
      cur_score = get_score(ciphertext, cur_key, alpha_key, n_gram, text_ngram_freq)
      new_score = get_score(ciphertext, new_key, alpha_key, n_gram, text_ngram_freq)
      if new_score == cur_score:
        continue
      if np.exp(new_score - cur_score) > np.random.rand():
        cur_key = new_key
      if new_score > best_score:
        best_score = new_score
        best_key = cur_key
  
  return best_key, best_score

In [368]:
n_gram = 2
tries = 3
iters = 20000
ciphertext = ciphertext_3

best_key, best_score = mcmc_decrypting(ciphertext, alpha_key, n_gram, text_ngram_freq, tries, iters)

In [369]:
print(f'Скор с исходным ключом шифрования: {round(get_score(ciphertext, key, alpha_key, n_gram, text_ngram_freq), 2)}')
print(f'Лучший скор в результате обучения: {round(best_score, 2)}')

Скор с исходным ключом шифрования: 3432.64
Лучший скор в результате обучения: 3442.48


In [370]:
# Получаем расшифрованный текст
decrypt_text_with_key(ciphertext, best_key, alpha_key)

'княжна марья возвратилась в свою комнату с чрустным испучанным выражением которое редко покидало ее и делало ее некрасивое болезненное лихо еше более некрасивым села за свой письменный стол уставленный миниатюрными портретами и заваленный тетрадями и кничами княжна была столь же беспорядогна как отех ее порядоген она положила тетрадь чеометрии и нетерпеливо распегатала письмо'

**Вывод:**
Как видим, в результате обучения зашифрованный текст получилось полностью расшифровать и прочитать.

## 4. Расшифровка сообщения
Задача — расшифровать сообщение:
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏

In [371]:
ciphertext_4 = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'

In [381]:
ciphertext_freq = descending_key(Counter(ciphertext_4))
print(ciphertext_freq)

['↹', '←', '⇛', '↟', '⇒', '↝', '⇴', '↨', '⇠', '⇯', '↷', '⇌', '⇊', '⇞', '⇈', '⇷', '↤', '↳', '↾', '↙', '⇙', '↲', '↞', '⇆', '⇰', '⇸', '↘', '⇏']


In [382]:
converted = decrypt_text_with_key(ciphertext_4, ciphertext_freq, alpha_key)
print(converted)

олна рд раяасо иевьтнуидг ана пемса иевьтнуидг соклс б шсеые леезжоиач кесевдг ноыке пвемастсу лкевоо рлоые рд рло ляонтна пвтрануие а пенбмасо ьтклаьтнуидг зтнн йт пелнояиоо мосровсео йтятиао кбвлт хесч кеиомие ч иамоые ио езожтю


In [385]:
tries = 5
iters = 30000
best_key_4, _ = mcmc_decrypting(converted, alpha_key, n_gram, text_ngram_freq, tries, iters)

In [391]:
# Расшифрованное сообщение
message = decrypt_text_with_key(converted, best_key_4, alpha_key)
print(message)

если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж


In [392]:
original_message = 'если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю'

In [393]:
print(f'Точность определения: {round(get_accuracy(original_message, message), 2)}')

Точность определения: 0.96


**Комментарий:** у нас получилось почти идеально расшифровать зашифрованное сообщение. Для 100% точности не хватает 1ой перестановки

## 5. Пробуем использовать n-граммы
Вопросы: а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты?

Попробуем добиться 100% точности расшифровки сообщения с помощью триграмм

In [390]:
n_gram = 3
tries = 3
iters = 20000
text_ngram_freq = Counter(ngrams(list(text_ru_cleaned), n_gram))
best_key_5, _ = mcmc_decrypting(converted, alpha_key, n_gram, text_ngram_freq, tries, iters)

In [394]:
message = decrypt_text_with_key(converted, best_key_5, alpha_key)
print(message)

если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю


In [395]:
print(f'Точность определения: {round(get_accuracy(original_message, message), 2)}')

Точность определения: 1.0


Триграммы помогли нам полностью расшифровать сообщение без потери качества. Посмотрим на эффективность использования n-грамм на примере текста из 1-го пункта.

In [396]:
import time

In [403]:
def print_ngram_scores(original_text: str, ciphertext: str, alpha_key: list, n_grams: list, tries: int, iters_list: list):
  
  for n_gram in n_grams:
    print(f'CHECKING RESULTS FOR N_GRAM = {n_gram} with {tries} tries')
    text_ngram_freq = Counter(ngrams(list(text_ru_cleaned), n_gram))

    for iters in iters_list:
      accuracy = []
      execution_time = []
      for _ in range(tries):        
        start_time = time.time()
        best_key, _ = mcmc_decrypting(ciphertext, alpha_key, n_gram, text_ngram_freq, 1, iters)
        decrypted_text = decrypt_text_with_key(ciphertext, best_key, alpha_key)
        accuracy.append(get_accuracy(original_text, decrypted_text))
        execution_time.append(time.time() - start_time)
      print(f'For {iters} iters: mean accuracy {round(np.mean(accuracy), 3)} in avarege {np.mean(execution_time)} sec')

    print('==============================================')

In [405]:
n_grams = [2, 3, 4]
tries = 5
iters_list = [100, 500, 1000, 3000, 5000, 8000, 12000, 20000]
print_ngram_scores(example_text_3, ciphertext_3, alpha_key, n_grams, tries, iters_list)

CHECKING RESULTS FOR N_GRAM = 2 with 5 tries
For 100 iters: mean accuracy 0.37 in avarege 0.14518232345581056 sec
For 500 iters: mean accuracy 0.47 in avarege 0.7091013431549072 sec
For 1000 iters: mean accuracy 0.485 in avarege 1.4279993057250977 sec
For 3000 iters: mean accuracy 0.743 in avarege 4.280170822143555 sec
For 5000 iters: mean accuracy 0.86 in avarege 7.12191333770752 sec
For 8000 iters: mean accuracy 0.841 in avarege 11.401575469970703 sec
For 12000 iters: mean accuracy 0.868 in avarege 17.100476789474488 sec
For 20000 iters: mean accuracy 0.824 in avarege 28.515150594711304 sec
CHECKING RESULTS FOR N_GRAM = 3 with 5 tries
For 100 iters: mean accuracy 0.462 in avarege 0.19246377944946289 sec
For 500 iters: mean accuracy 0.505 in avarege 0.9744288921356201 sec
For 1000 iters: mean accuracy 0.691 in avarege 1.9775525093078614 sec
For 3000 iters: mean accuracy 0.824 in avarege 5.905860853195191 sec
For 5000 iters: mean accuracy 0.998 in avarege 9.824980354309082 sec
For 8000

**Выводы:** на примере текста из первого пункта видим, что чем больше n-грамма используется, тем эффективнее модель. Не смотря на то, что большая n-грамма в среднем требует больше времени на исполнение 1 итерации, это компенсируется тем, что точность расшифровки получается выше за меньшее количество итераций (и времени соответственно).

* биграммы - точность 82% за 20к итераций (28,5 сек)
* триграммы - точность 100% менее чем за 12к итераций (23,6 сек)
* 4-граммы - точность 100% менее чем за 5к итераций (10,9 сек)

Вероятно, проблемы будут возникать при небольших текстах, так как число n-грамм будет формироваться меньше.

## 6. Какие вы можете придумать применения для этой модели?

1. Возможно, данную модель можно применить в социальной сфере, например, для анализа каких-либо общих паттернов поведения людей, если вместо символа использовать какое-либо событие, совершенное конкретным человеком.
2. Возможно использование данного метода, чтобы привести данные к единному ввиду, например, разные характеристики по наименованию привести к единным по сути. И уже обработанные данные использовать в машинном обучении.
